# IFC Exploration (Day 1 Morning)
Learn IFC structure + see your building model

In [ ]:
import ifcopenshell
import ifcopenshell.util.element as util
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

## 1. Load Model

In [ ]:
model = ifcopenshell.open('../00_data/ifc_models/01_Duplex_Apartment.ifc')
print(f"✓ Loaded {model.schema}")
print(f"✓ Project: {model.by_type('IfcProject')[0].Name}")

## 2. Quick Visual - What's in the Building?

In [ ]:
# Count element types
element_counts = {}
for product in model.by_type('IfcProduct'):
    etype = product.is_a()
    element_counts[etype] = element_counts.get(etype, 0) + 1

# Show top 10 as bar chart
top_types = sorted(element_counts.items(), key=lambda x: x[1], reverse=True)[:10]
names, counts = zip(*top_types)

plt.figure(figsize=(10, 5))
plt.barh(names, counts, color='steelblue')
plt.xlabel('Count')
plt.title('IFC Element Types in Model')
plt.tight_layout()
plt.show()

print(f"\nTotal: {sum(element_counts.values())} elements")

## 3. Explore Doors

In [ ]:
doors = model.by_type('IfcDoor')
print(f"Found {len(doors)} doors\n")

for door in doors[:3]:  # First 3 doors
    print(f"🚪 {door.Name or 'Unnamed'}")
    print(f"   ID: {door.GlobalId}")
    
    # Width from direct attribute
    if door.OverallWidth:
        width = door.OverallWidth
        print(f"   Width: {width:.0f} mm")
    
    # Properties from property sets
    psets = util.get_psets(door)
    print(f"   Property Sets: {list(psets.keys())}")
    print()

## 4. Simple 3D View - Door Locations

In [ ]:
# Extract door positions (simplified - just plot their locations)
door_positions = []

for door in doors:
    # Get placement (simplified extraction)
    if hasattr(door, 'ObjectPlacement'):
        placement = door.ObjectPlacement
        if hasattr(placement, 'RelativePlacement'):
            loc = placement.RelativePlacement.Location
            if hasattr(loc, 'Coordinates'):
                coords = loc.Coordinates
                door_positions.append(coords)

if door_positions:
    positions = np.array(door_positions)
    
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(positions[:, 0], positions[:, 1], positions[:, 2], 
               c='brown', s=100, marker='s', label='Doors')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z (Height)')
    ax.set_title('Door Locations in Building')
    ax.legend()
    plt.show()
    
    print(f"Plotted {len(positions)} door locations")
else:
    print("Could not extract door positions (vendor-specific placement)")

## 5. YOUR TURN: Explore Windows or Walls
Try extracting properties from windows or checking wall thickness

In [ ]:
# Your code here:
windows = model.by_type('IfcWindow')
print(f"Found {len(windows)} windows")

# TODO: Explore their properties, dimensions, etc.

## Key Takeaways

1. IFC files contain **typed entities** (IfcDoor, IfcWall, etc.)
2. Properties are in **PropertySets** (vendor chaos - different names!)
3. Some data is **direct attributes** (OverallWidth), some in psets
4. **Next step:** Build compliance checks in `app_simple.py`